In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np

/usr/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
SAVED_MODEL_PATH = "/mnt/Data/tmp/resnet/standard_56" # Saved Plastic ResNet checkpoint
TO_SAVE_PATH = SAVED_MODEL_PATH + "/mnist" # Where to save current model
BATCH_SIZE = 32
EPOCHS = 200
LEARN_RATE = 5e-5

In [4]:
def train_input_fn(mnist):
    train_dataset = tf.data.Dataset.from_tensor_slices((mnist.train.images,
                                                        mnist.train.labels))

    map_fn = lambda img, lbl: (tf.image.resize_images(tf.reshape(img, [28, 28, 1]),
                                                      (32, 32)), lbl)
    train_dataset = train_dataset.map(map_fn).shuffle(10000).apply(
        tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE))

    return train_dataset

In [5]:
def test_input_fn(mnist):
    test_dataset = tf.data.Dataset.from_tensor_slices((mnist.test.images,
                                                       mnist.test.labels))

    map_fn = lambda img, lbl: (tf.image.resize_images(tf.reshape(img, [28, 28, 1]),
                                                      (32, 32)), lbl)
    test_dataset = test_dataset.map(map_fn).shuffle(10000).apply(
        tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE))

    return test_dataset

In [6]:
mnist = tf.contrib.learn.datasets.load_dataset('mnist')

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [7]:
def resnet_layer(inputs, filters=16, kernel_size=3, strides=1, activation=tf.nn.relu,
                 batch_normalization=True, trainable=False):
    # Function from keras's example for Cifar10:
    # https://github.com/keras-team/keras/blob/master/examples/cifar10_resnet.py
    x = tf.layers.conv2d(inputs=inputs, filters=filters, kernel_size=kernel_size,
                         strides=strides, padding='same', trainable=trainable)
    if batch_normalization:
        x = tf.layers.batch_normalization(x, trainable=trainable)
    if activation is not None:
        x = activation(x)
    return x

In [8]:
def resnet_model(features, labels, mode):
    # Function from keras's example for Cifar10:
    # https://github.com/keras-team/keras/blob/master/examples/cifar10_resnet.py
    filters = 16
    num_res_blocks = 9
    
    features = tf.cast(features, dtype=tf.float32)
    x = resnet_layer(inputs=features, trainable=True)
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:
                strides = 2
            y = resnet_layer(inputs=x, filters=filters, strides=strides,
                             trainable=True)
            y = resnet_layer(inputs=y, filters=filters, activation=None,
                             trainable=True)
            if stack > 0 and res_block == 0:
                x = resnet_layer(inputs=x, filters=filters, kernel_size=1,
                                 strides=strides, activation=None,
                                 batch_normalization=False, trainable=True)
            x = tf.nn.relu(x + y)
        filters *= 2
    
    x = tf.layers.average_pooling2d(x, pool_size=8, strides=8)
    x = tf.layers.flatten(x)
    
    #x = tf.layers.dense(inputs=x, units=10)
    w = tf.get_variable('Variable', (int(x.shape[1]), 10),
                        initializer=tf.truncated_normal_initializer)
    b = tf.get_variable('Variable_1', (10,),
                        initializer=tf.truncated_normal_initializer)
    x = tf.matmul(x, w) + b
    
    classes = tf.argmax(input=x, axis=1)
    correct_prediction = tf.equal(tf.cast(classes, tf.uint8), labels)
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) * 100
    acc = tf.identity(acc, name='accuracy_tensor')
    
    predictions = {'classes': classes, 'accuracy': acc}
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
        
    onehot_labels = tf.one_hot(tf.cast(labels, tf.int32), 10)
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=onehot_labels, logits=x)
    loss = tf.reduce_mean(loss)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.contrib.estimator.TowerOptimizer(tf.train.AdamOptimizer(
            LEARN_RATE))
        train_op = optimizer.minimize(loss=loss,
                                      global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {'accuracy': tf.metrics.accuracy(labels=labels,
                                                       predictions=classes)}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss,
                                      eval_metric_ops=eval_metric_ops)

In [9]:
tensors_to_log = {'accuracy': 'accuracy_tensor'}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)

In [10]:
vars_warm = []
for var in ('bias', 'kernel'):
    vars_warm.append("conv2d/{}".format(var))
    for num in range(1, 57):
        vars_warm.append("conv2d_{}/{}".format(num, var))
for var in ('beta', 'gamma', 'moving_mean', 'moving_variance'):
    vars_warm.append("batch_normalization/{}".format(var))
    for num in range(1, 55):
        vars_warm.append("batch_normalization_{}/{}".format(num, var))
#vars_warm.append("Variable")
#vars_warm.append("Variable_1")
vars_warm += ['beta1_power', 'beta2_power']

In [11]:
warm_start = tf.estimator.WarmStartSettings(
    ckpt_to_initialize_from=SAVED_MODEL_PATH, vars_to_warm_start=vars_warm)

In [12]:
model = tf.estimator.Estimator(model_fn=tf.contrib.estimator.replicate_model_fn(
    resnet_model), model_dir=TO_SAVE_PATH, warm_start_from=warm_start)

Instructions for updating:
Please use `tf.contrib.distribute.MirroredStrategy` instead.
INFO:tensorflow:Replicating the `model_fn` across ['/device:GPU:0'].  Variables are going to be placed on ['/device:GPU:0'].  Consolidation device is going to be /device:GPU:0.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/mnt/Data/tmp/resnet/standard_56/mnist', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0693e07b70>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [16]:
max_steps = int((mnist.train.images.shape[0] / BATCH_SIZE) * EPOCHS)
model.train(input_fn=lambda:train_input_fn(mnist), hooks=[logging_hook],
            max_steps=max_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/mnt/Data/tmp/resnet/standard_56', vars_to_warm_start=['conv2d/bias', 'conv2d_1/bias', 'conv2d_2/bias', 'conv2d_3/bias', 'conv2d_4/bias', 'conv2d_5/bias', 'conv2d_6/bias', 'conv2d_7/bias', 'conv2d_8/bias', 'conv2d_9/bias', 'conv2d_10/bias', 'conv2d_11/bias', 'conv2d_12/bias', 'conv2d_13/bias', 'conv2d_14/bias', 'conv2d_15/bias', 'conv2d_16/bias', 'conv2d_17/bias', 'conv2d_18/bias', 'conv2d_19/bias', 'conv2d_20/bias', 'conv2d_21/bias', 'conv2d_22/bias', 'conv2d_23/bias', 'conv2d_24/bias', 'conv2d_25/bias', 'conv2d_26/bias', 'conv2d_27/bias', 'conv2d_28/bias', 'conv2d_29/bias', 'conv2d_30/bias', 'conv2d_31/bias', 'conv2d_32/bias', 'conv2d_33/bias', 'conv2d_34/bias', 'conv2d_35/bias', 'conv2d_36/bias', 'conv2d_37/bias', 'conv2d_38/bias', 'conv2d_39/bias', 'conv2d_40/bias', 'conv2d_41/bias', 'conv2d_42/bias', 'conv2d_43/bi

INFO:tensorflow:Warm-starting from: ('/mnt/Data/tmp/resnet/standard_56',)
INFO:tensorflow:Warm-starting variable: conv2d/bias; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable conv2d/bias:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with conv2d/bias
INFO:tensorflow:Warm-starting variable: conv2d/bias/Adam; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable conv2d/bias/Adam:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with conv2d/bias/Adam
INFO:tensorflow:Warm-starting variable: conv2d/bias/Adam_1; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable conv2d/bias/Adam_1:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with conv2d/bias/Adam_1
INFO:tensorflow:Warm-starting variable: conv2d_1/bias; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable conv2d_1/bias:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with conv2d_1/bias
INFO:tensorflow:Warm-starting variable: conv2d_1/bias/Adam; prev_var_name: Unchanged
INFO:tensorflow:Initial

INFO:tensorflow:Initialize variable conv2d_12/bias/Adam_1:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with conv2d_12/bias/Adam_1
INFO:tensorflow:Warm-starting variable: conv2d_13/bias; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable conv2d_13/bias:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with conv2d_13/bias
INFO:tensorflow:Warm-starting variable: conv2d_13/bias/Adam; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable conv2d_13/bias/Adam:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with conv2d_13/bias/Adam
INFO:tensorflow:Warm-starting variable: conv2d_13/bias/Adam_1; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable conv2d_13/bias/Adam_1:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with conv2d_13/bias/Adam_1
INFO:tensorflow:Warm-starting variable: conv2d_14/bias; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable conv2d_14/bias:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with conv2d_14/bias
INFO:tensorflow:

INFO:tensorflow:Warm-starting variable: conv2d_25/bias/Adam_1; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable conv2d_25/bias/Adam_1:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with conv2d_25/bias/Adam_1
INFO:tensorflow:Warm-starting variable: conv2d_26/bias; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable conv2d_26/bias:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with conv2d_26/bias
INFO:tensorflow:Warm-starting variable: conv2d_26/bias/Adam; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable conv2d_26/bias/Adam:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with conv2d_26/bias/Adam
INFO:tensorflow:Warm-starting variable: conv2d_26/bias/Adam_1; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable conv2d_26/bias/Adam_1:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with conv2d_26/bias/Adam_1
INFO:tensorflow:Warm-starting variable: conv2d_27/bias; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable conv2d_27/bias

INFO:tensorflow:Initialize variable conv2d_38/bias/Adam:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with conv2d_38/bias/Adam
INFO:tensorflow:Warm-starting variable: conv2d_38/bias/Adam_1; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable conv2d_38/bias/Adam_1:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with conv2d_38/bias/Adam_1
INFO:tensorflow:Warm-starting variable: conv2d_39/bias; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable conv2d_39/bias:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with conv2d_39/bias
INFO:tensorflow:Warm-starting variable: conv2d_39/bias/Adam; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable conv2d_39/bias/Adam:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with conv2d_39/bias/Adam
INFO:tensorflow:Warm-starting variable: conv2d_39/bias/Adam_1; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable conv2d_39/bias/Adam_1:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with conv2d_39/bias/Adam_1

...

INFO:tensorflow:Initialize variable batch_normalization_52/gamma/Adam_1:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with batch_normalization_52/gamma/Adam_1
INFO:tensorflow:Warm-starting variable: batch_normalization_53/gamma; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_53/gamma:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with batch_normalization_53/gamma
INFO:tensorflow:Warm-starting variable: batch_normalization_53/gamma/Adam; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_53/gamma/Adam:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with batch_normalization_53/gamma/Adam
INFO:tensorflow:Warm-starting variable: batch_normalization_53/gamma/Adam_1; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_53/gamma/Adam_1:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with batch_normalization_53/gamma/Adam_1
INFO:tensorflow:Warm-starting variable: batch_normalization_54/ga

INFO:tensorflow:Warm-starting variable: batch_normalization_25/moving_mean; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_25/moving_mean:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with batch_normalization_25/moving_mean
INFO:tensorflow:Warm-starting variable: batch_normalization_26/moving_mean; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_26/moving_mean:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with batch_normalization_26/moving_mean
INFO:tensorflow:Warm-starting variable: batch_normalization_27/moving_mean; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_27/moving_mean:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with batch_normalization_27/moving_mean
INFO:tensorflow:Warm-starting variable: batch_normalization_28/moving_mean; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_28/moving_mean:0 from checkpoint /mnt/Data/tmp/resnet/

INFO:tensorflow:Initialize variable batch_normalization_1/moving_variance:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with batch_normalization_1/moving_variance
INFO:tensorflow:Warm-starting variable: batch_normalization_2/moving_variance; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_2/moving_variance:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with batch_normalization_2/moving_variance
INFO:tensorflow:Warm-starting variable: batch_normalization_3/moving_variance; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_3/moving_variance:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with batch_normalization_3/moving_variance
INFO:tensorflow:Warm-starting variable: batch_normalization_4/moving_variance; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_4/moving_variance:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with batch_normalization_4/moving_variance
INFO:tensorflow:

INFO:tensorflow:Initialize variable batch_normalization_31/moving_variance:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with batch_normalization_31/moving_variance
INFO:tensorflow:Warm-starting variable: batch_normalization_32/moving_variance; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_32/moving_variance:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with batch_normalization_32/moving_variance
INFO:tensorflow:Warm-starting variable: batch_normalization_33/moving_variance; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_33/moving_variance:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with batch_normalization_33/moving_variance
INFO:tensorflow:Warm-starting variable: batch_normalization_34/moving_variance; prev_var_name: Unchanged
INFO:tensorflow:Initialize variable batch_normalization_34/moving_variance:0 from checkpoint /mnt/Data/tmp/resnet/standard_56 with batch_normalization_34/moving_variance
INFO:

INFO:tensorflow:global_step/sec: 1.96127
INFO:tensorflow:accuracy = 93.75 (50.987 sec)
INFO:tensorflow:loss = 0.21215607, step = 800 (50.987 sec)
INFO:tensorflow:global_step/sec: 1.98473
INFO:tensorflow:accuracy = 96.875 (50.385 sec)
INFO:tensorflow:loss = 0.16136551, step = 900 (50.385 sec)
INFO:tensorflow:global_step/sec: 2.04505
INFO:tensorflow:accuracy = 93.75 (48.899 sec)
INFO:tensorflow:loss = 0.19556665, step = 1000 (48.898 sec)
INFO:tensorflow:global_step/sec: 2.09907
INFO:tensorflow:accuracy = 100.0 (47.640 sec)
INFO:tensorflow:loss = 0.059415217, step = 1100 (47.640 sec)
INFO:tensorflow:global_step/sec: 2.13821
INFO:tensorflow:accuracy = 96.875 (46.768 sec)
INFO:tensorflow:loss = 0.05929832, step = 1200 (46.768 sec)
INFO:tensorflow:Saving checkpoints for 1288 into /mnt/Data/tmp/resnet/standard_56/mnist/model.ckpt.
INFO:tensorflow:global_step/sec: 2.1313
INFO:tensorflow:accuracy = 96.875 (46.919 sec)
INFO:tensorflow:loss = 0.09116112, step = 1300 (46.919 sec)
INFO:tensorflow:g

In [17]:
print(model.evaluate(input_fn=lambda:test_input_fn(mnist)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-29-09:41:02
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /mnt/Data/tmp/resnet/standard_56/mnist/model.ckpt-1718
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-29-09:41:34
INFO:tensorflow:Saving dict for global step 1718: accuracy = 0.9657452, global_step = 1718, loss = 0.11152244
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1718: /mnt/Data/tmp/resnet/standard_56/mnist/model.ckpt-1718
{'accuracy': 0.9657452, 'loss': 0.11152244, 'global_step': 1718}
